In [16]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
from IPython.display import HTML
import time

In [17]:
browser = Browser('chrome')
base_url = 'https://www.indeed.com'
url = 'https://www.indeed.com/jobs?q=junior+data+analyst&l=&vjk=3431753fef1de31b'
browser.visit(url)

In [18]:
html = browser.html
soup = BeautifulSoup(html,'html.parser')

In [19]:
results = soup.find('ul', class_='jobsearch-ResultsList css-0')

In [20]:
jobs = results.find_all('td', class_='resultContent')

In [21]:
job_list = []
exclude_words = ["a", "an", "the", "at", "by", "for", "in", "of", "on", "to", "with", "and", "but", "or", "so", "he", "she", "it", "they", "them", "you", "your", "we", "us", "our", "i", "me", "my", "mine", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "do", "does", "did", "will", "would", "should", "could", "may", "might", "must", "thing", "stuff", "something", "anything", "everything", "nothing", "one", "ones", "some", "many", "few","their"]


In [ ]:
def count_words(text, exclude=[]):
    words = text.split()
    words = [word.lower() for word in words if word.lower() not in exclude]
    word_counts = {word: words.count(word) for word in set(words)}
    return word_counts

In [22]:
while True:
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    results = soup.find('ul', class_='jobsearch-ResultsList css-0')
    jobs = results.find_all('td', class_='resultContent')
    for job in jobs:
        title = job.div.h2.a.text
        company = job.find('span', class_='companyName').text
        location = job.find('div', class_='companyLocation').text
        estimate = job.find('span', class_='estimated-salary')
        if estimate is not None:
            salary = estimate.text
        else:
            salary = 'N/A'
       

        link = job.find('a')['href']
        absolute_link = urljoin(base_url, link)        
        clickable_link = '<a href="' + absolute_link + '">Link</a>'

        browser.visit(absolute_link)
        

        
        job_dict={
            'Title':title,
            'Company':company,
            'Location':location,
            'Salary':salary,
            'Link':clickable_link
        }
        job_list.append(job_dict)
    button = browser.find_by_css('a[data-testid="pagination-page-next"]')
    if button:
        button.click()
        time.sleep(5)
    else:
        break

AttributeError: 'WebDriver' object has no attribute 'find_by_class'

In [23]:
browser.quit()

In [ ]:
df = pd.DataFrame(job_list)


In [ ]:
#pd.set_option('display.max_colwidth',None)

In [ ]:
df['Link'] = df['Link'].astype(str)
df_html = df.to_html(render_links=True, escape=False)

# To display the DataFrame with clickable links in a Jupyter notebook:
HTML(df_html)

,Title,Company,Location,Salary,Link
0,Junior Data Analyst,"Zartico, Inc.",Remote,Estimated $68.1K - $86.2K a year,Link
1,Junior Business Analyst,aezion inc,Remote,N/A,Link
2,Junior Data Analyst,Affiliated Monitoring,"Union, NJ 07083",N/A,Link
3,Junior Data Analyst,Talentheed Inc,"Hybrid remote in East Pittsburgh, PA 15112",N/A,Link
4,Junior Data Analyst,Cobbs Creek Healthcare,"Newtown Square, PA 19073",N/A,Link
5,Junior Data Analyst,Aptive,Remote+1 location,N/A,Link
6,Junior Data Strategy Analyst (JDSA),National Capitol Contracting (NCC),"Kittery, ME+2 locations",Estimated $88.3K - $112K a year,Link
7,Junior Data Analyst,Zartico,"Salt Lake City, UT 84101 (People's Freeway area)",Estimated $65.1K - $82.4K a year,Link
8,Data Analyst I,"Innovasystems International, LLC","Alexandria, VA",N/A,Link
9,Junior Data Analyst,TG Administration LLC,"Hybrid remote in Portland, OR 97205",N/A,Link


In [ ]:
#pd.set_option('display.max_colwidth',None)

# Remote Jobs

In [ ]:
remote_rows=df[df['Location'].str.contains('Remote')]

In [ ]:
display(HTML(remote_rows.to_html(render_links=True, escape=False)))

,Title,Company,Location,Salary,Link
0,Junior Data Analyst,"Zartico, Inc.",Remote,Estimated $68.1K - $86.2K a year,Link
1,Junior Business Analyst,aezion inc,Remote,N/A,Link
5,Junior Data Analyst,Aptive,Remote+1 location,N/A,Link
14,Remote Junior Data Analyst,Utilisave,"Remote in New York, NY 10001",N/A,Link
15,Junior Business Analyst,BPA Services,Remote,N/A,Link
19,Junior Data Analyst,IntelliBridge LLC,"Remote in Fredericksburg, VA 22407",Estimated $58.4K - $74K a year,Link
24,Junior Database Analyst,EloTouch,"Remote in Richardson, TX 75081",Estimated $92.8K - $117K a year,Link
28,Jr. Business/Data Systems Analyst - BDSA1601,Embedded Techniques,Remote,N/A,Link
38,Junior Business Analyst/Remote,Wabtec,"Remote in Duncan, SC 29334",Estimated $68.9K - $87.2K a year,Link
47,Junior Data Analyst,General Dynamics Information Technology,Remote in Virginia+1 location,Estimated $93.7K - $119K a year,Link
